In [1]:
import sys
sys.path.insert(0, '/home/christo/notebooks/christo/prod/')
sys.path.insert(0, '/home/christo/notebooks/christo/company_maintenance_scripts/')
import re
import pandas as pd
from NedbankSpendPy import  client
import time

In [2]:
from nedbank_companies_addition_lib import maxindexfn, mongo_regex_searchstring
from nedbank_companies_addition_lib import nedbankcompany_class_check, classlookupfn, input_dict_fn, mongo_regex_searchstring_row
from nedbank_companies_addition_lib import  nedbank_companies_db, regex_searchstring

In [3]:
def agg_fn(collection):
    return pd.DataFrame(
    list(collection.aggregate([{'$match':
                                                        {
                                                            'companyname':
                                                            {
                                                                '$exists':True
                                                            }
                                                        }
                                                       },
                                                       {'$project':{'companyname':1,
                                                                    'companyindex':1,
                                                                   'TransactionAmount':1}},
                                                       {
                                                           '$group':
                                                           {
                                                               '_id':
                                                               {
                                                                   'companyname':'$companyname',
                                                                   'companyindex':'$companyindex'
                                                               },
                                                               "transactioncount":
                                                               {
                                                                   '$sum': 1
                                                               },
                                                               "transactionamount":
                                                               {
                                                                   '$sum':'$TransactionAmount'
                                                               }
                                                           }
                                                       },
                                                       {'$project':{'_id':0,'companyname':'$_id.companyname',
                                                                    'companyindex':'$_id.companyindex',
                                                                   'transactioncount':1,
                                                                   'transactionamount':1}}
                                                      ], allowDiskUse=True)))



In [53]:
def franchise_agg_fn(collection):
    return pd.DataFrame(
    list(collection.aggregate([{'$match':
                                                        {
                                                            'companyname':
                                                            {
                                                                '$exists':True
                                                            }
                                                        }
                                                       },
                                                       {'$project':{'franchisename':1,
                                                                    'Dedupegroup':1,
                                                                    'TransactionDate':1,
#                                                                     'companyindex':1,
                                                                   'TransactionAmount':1}},
                                                       {
                                                           '$group':
                                                           {
                                                               '_id':
                                                               {
                                                                   'dedupegroup':'$Dedupegroup',
                                                                   'transactiondate':'$TransactionDate',
#                                                                    'franchisename':'$franchisename'
                                                               },
                                                               "franchisenameset":{"$addToSet":'$franchisename'},
                                                               "transactioncount":
                                                               {
                                                                   '$sum': 1
                                                               },
                                                               "transactionamount":
                                                               {
                                                                   '$sum':'$TransactionAmount'
                                                               }
                                                           }
                                                       },
                               {
                                   '$match':
                                   {
                                       'franchisenameset':{ '$in': [ 'PNP ON NICOL' ] }
                                   }
                                   
                               },
                                                       {'$project':{'_id':0,'dedupegroup':'$_id.dedupegroup',
                                                                    'franchisenameset':'$franchisenameset',
#                                                                     'n_uniqueclients':{'$size': "$uniqueclientlist"},
                                                                    'transactiondate':'$_id.transactiondate',
                                                                   'transactioncount':1,
                                                                   'transactionamount':1}}
                                                      ], allowDiskUse=True)))



In [54]:
collectionlist=['clientswipes_201701','clientswipes_201702','clientswipes_201703',
                'clientswipes_201704','clientswipes_201705','clientswipes_201706',
                'clientswipes_201707','clientswipes_201708','clientswipes_201709',
                'clientswipes_201710','clientswipes_201711','clientswipes_201712',
                'clientswipes_201801','clientswipes_201802','clientswipes_201803',
                'clientswipes_201804','clientswipes_201805','clientswipes_201806',
                'clientswipes_201807','clientswipes_201808','clientswipes_201809',
                'clientswipes_201810','clientswipes_201811','clientswipes_201812',                
                'clientswipes_201901','clientswipes_201902','clientswipes_201903',
                'clientswipes_201904','clientswipes_201905','clientswipes_201906',
                'clientswipes_201907','clientswipes_201908','clientswipes_201909',
                'clientswipes_201910','clientswipes_201911','clientswipes_201912']

In [55]:
from datetime import datetime 

In [56]:
collectionname=collectionlist[0]
collection=client.behaviourdb[collectionname]
df=franchise_agg_fn(collection)

In [37]:
df

,dedupegroup,franchisenameset,transactionamount,transactioncount,transactiondate
0,1.100063e+11,"[Fournos Dunkeld, Checkers BalfourPark]",-150.19,2,2017-01-05
1,1.100063e+11,"[Fournos Dunkeld, WOOLWORTHS PARKTOWN, PAYU: D...",-1739.99,4,2017-01-26
2,1.100066e+11,[Fournos Dunkeld],-70.00,1,2017-01-15
3,1.100066e+11,[Fournos Dunkeld],-148.00,1,2017-01-17
4,1.100076e+11,[Fournos Dunkeld],-370.00,1,2017-01-14
5,1.100076e+11,"[Fournos Dunkeld, Clicks Hyde Park]",-352.05,2,2017-01-27
6,1.100078e+11,[Fournos Dunkeld],-59.80,1,2017-01-25
7,1.100102e+11,[Fournos Dunkeld],-121.80,1,2017-01-18
8,1.100106e+11,[Fournos Dunkeld],-381.50,1,2017-01-21
9,1.100138e+11,"[Fournos Dunkeld, WOOLWORTHS GREENSIDE, DOPPIO...",-2424.70,3,2017-01-28


In [27]:
list(df)

['dedupegroup',
 'franchisenameset',
 'transactionamount',
 'transactioncount',
 'transactiondate']

In [38]:
result_dict={}
for franchisenameset in list(df.franchisenameset):
    for franchisename in franchisenameset:
        if franchisename in result_dict:
            result_dict[franchisename]+=1
        else:
            result_dict[franchisename]=1

In [39]:
result_dict

{'ABSOLUTE PETS-HYDE P': 1,
 'ACKERMANS SAND': 1,
 'ACSA JIA': 2,
 'ADIDAS MALL OF AFRICA HOM': 1,
 'ALZU PETROPORT N': 1,
 'AMAZON DIGITAL DWNLDS': 1,
 'AMAZON SERVICES-KINDLE': 1,
 'Amazon Services-Kindle': 1,
 'BP BIRDHAVEN': 1,
 'BP GALLO MANOR': 1,
 'BP LINDEN PA': 1,
 'BP MALIBONGWE': 1,
 'BP OLIVEDALE': 1,
 'BP RIDGE OAS': 1,
 'BP SMIT STREET': 1,
 'BP WILLIAM NIC': 1,
 'BP ZOO LAKE GARAGE': 1,
 'BWH FOURWAYS': 1,
 'BWH FOURWAYS          YS': 1,
 'BWH RIVONIA': 2,
 'CALTEX CEDAR ROAD': 1,
 'CALTEX LOUIS BOTHA': 1,
 'CALTEX OAKLANDS': 2,
 'CALTEX ROSEBANK': 1,
 'CALTEX WENDYWOOD SER': 1,
 'CAPE UNION MART- HYDE AR': 1,
 'CHECKERS HYPER SANDTON': 1,
 'CNA BLAIRGOWRIE': 1,
 'CNA KILLARNEY': 2,
 'CNA SANDTON CITY': 2,
 'COMPUTICKET CALL CEN': 1,
 'COTTON ON KIDS ROSEBAN': 1,
 'COTTON ON ROSEBANK COC': 1,
 'COTTON ON SANDTON COC': 1,
 'COTTON ON SANDTON COC UAR': 1,
 'CRAZY PLASTICS RANDB': 1,
 'Checkers BalfourPark': 2,
 'Checkers Bryanston': 1,
 'Checkers Emmarentia': 5,
 'Checkers

In [47]:
sorted_result_dict = sorted(result_dict.items(), key=lambda kv: kv[1], reverse=True)

In [52]:
# list(sorted_result_dict)
[sorted_result_dict[ind][0] for ind in range(len(sorted_result_dict))]

['Fournos Dunkeld',
 'SPAR CRAIGHALL',
 'WOOLWORTHS HYDEPARK JH',
 'DUNKELD MICA',
 'UBER SA HELP.UBER.CO',
 'ITUNES.COM/BILL',
 'UBER BV',
 'Clicks Hyde Park',
 'WOOLWORTHS ROSEBANK   B',
 'PnP Crp Hydepark',
 'DIS-CHEM ATHOL OAKLANDS',
 'WOOLWORTHS HYDEPARK',
 'WOOLWORTHS VALLEY CENTNTR',
 'PnP Fam Craighall ParkARK',
 'WOOLWORTHS BLUBIRD CEN',
 'WOOLWORTHS ROSEBANK',
 'HUTTON HYDE PARK BP',
 'PNP HYDEPARK',
 'PNP ON NICOL',
 'Spar SuperA Norwod Spa',
 'WOOLWORTHS PARKTOWN',
 'WOOLWORTHS BENMORE GAR',
 'WOOLWORTHS VALLEY CENTR',
 'Checkers Emmarentia',
 'PNP CRAIGHALL PARK',
 'DUNKELD MICA HARDWAR',
 'FOOD LOVERS MARKET NORWOO',
 'PnP Hpr Norwood',
 'WOOLWORTHS PARKTOWN QUATE',
 'DIS-CHEM ROSEBANK',
 'WOOLWORTHS SANDTON CITY',
 'Clicks Valley Centre',
 'DIS-CHEM NORWOOD EMV',
 'PnP Crp On Nicol',
 'SPAR PARKVIEW',
 'MANGO AIRLINES',
 'WOOLWORTHS PARKWOOD JH',
 'DIS-CHEM ROSEBANK - CK',
 'DOPPIO ZERO',
 'WOOLWORTHS GRAYSTONE RON',
 'TOTAL ILLOVO',
 'ITUNES.COM/BIL',
 'WOOLWORTHS VICTO

In [ ]:
dn=datetime.now()
m=dn.month
d=dn.day
y=dn.year
savestr= f'{y:0}'+f'{m:02}'+f'{d:02}'
df=pd.DataFrame()
for collectionname in collectionlist:
#     print('-----------------------------------------------------------------------------')
    print('===== now doing collection: {collectionname} ====================================='.format(collectionname=collectionname))
    period=collectionname[-6:]
    print('Doing period: {period}'.format(period=period))
    collection=client.behaviourdb[collectionname]
    print('Collect data:')    
    agg_df=client_agg_fn(collection)
    print('Set index:')    
    agg_df.set_index(['companyindex','companyname'],inplace=True)
    print('Rename columns:')        
    agg_df.rename(columns={'transactionamount':'amount_{period}'.format(period=period),
                      'transactioncount':'count_{period}'.format(period=period)}, inplace=True)
    if df.empty:
        print('The history is still empty!')
        df=agg_df.copy()
    else:
        print('Merge new data with (non-empty) history:')        
        df=df.merge(agg_df, right_index=True,left_index=True, how='outer')
    print('Save (new) history:')
    df.to_csv(df.to_csv('/home/christo/data/companydb_aggregated_pivot_{savestr}.csv'.format(savestr=savestr)))
    

In [ ]:
savestr

In [ ]:
df.to_csv('/home/christo/data/companydb_aggregated_pivot_{savestr}.csv'.format(savestr=savestr))

In [ ]:
collection = client.lookups['aggregated_company_spend']
df.reset_index(inplace=True)
collection.insert_many(df.to_dict('records'))